<h1 align="center">Clustering Neighborhoods in Toronto</h1>
<h5 align="center">Coursera Course 9 Week-3 Assignment <b>Part-C</b>      Sagar Shah</h5>


(1) Using Beautifulsoup package to scrap html from wikipedia page for Toronto  
(2) Explore Neighborhoods in Toronto  
(3) Analyze Each neighbourhoods  
(4) Cluster Neighbourhoods  
(5) Examine and lable cluster by its distinct characteristics

#### 1. Installing and importing necessary packages and scrapping html to get Toronto neighbourhood info

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge folium=0.5.0 --yes

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen

import folium # map rendering library


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00 754.72 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   2.78 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  25.80 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.52 MB/s
folium-0.5.0-p 100% |################################| Time: 0

#### Get the Wikipedia package for Canada postal code

In [2]:
htmldoc = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#### Instantiate Beautiful Soup

In [3]:
soupobj = BeautifulSoup(htmldoc,"lxml")

#### Scrap the table for postal code from html page

In [4]:
toronto_table = soupobj.find_all('table',class_ = 'wikitable sortable')
toronto_table = toronto_table[0]
records = toronto_table.findAll("tr")


#### Output the table to CSV file

In [5]:
csvfile = open('Toronto_postcode.csv','wt',newline='')
writer = csv.writer(csvfile)


In [6]:
try:
   for row in records:
       csvrow = []
       for cell in row.findAll(['td','th']):
           csvrow.append(cell.get_text())
       writer.writerow(csvrow)
finally:
   csvfile.close()

#### Read CSV and create pandas Dataframe

In [7]:
df = pd.read_csv('Toronto_postcode.csv') 
df.shape

(289, 3)

In [8]:
df=df.rename(columns = {'Neighbourhood\n':'Neighbourhood'})
df['Neighbourhood']= df['Neighbourhood'].str.replace(r'\n', '')
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.shape

(212, 3)

### Group all the neighbourhoods in same Borough

In [9]:
df = df.groupby(['Borough','Postcode'], as_index=False, sort=False).agg(', '.join)


### Update Neighbourhood with same value from Borough when it is Not Assigned

In [10]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'] , df['Neighbourhood'])
df.head(1)

,Borough,Postcode,Neighbourhood
0,North York,M3A,Parkwoods


#### Provide the shape of Dataframe

In [11]:
df.shape

(103, 3)

Combining 2 columns to form Address field, however it is not needed anymore since geocode API which needs it; does not work anymore

In [12]:
df["Address"]= df['Borough'] + ", " + df['Postcode'] 
df.head(1)

,Borough,Postcode,Neighbourhood,Address
0,North York,M3A,Parkwoods,"North York, M3A"


Following part is not needed anymore since I kept on getting Over query Limit error

In [13]:
#!conda install -c conda-forge geocoder --yes
#import geocoder

In [14]:
#g = geocoder.google('Mountain View, CA')
#g.latlong
#g

#### Getting Postal code to coodinates file into Dataframe 

In [15]:
!wget -q -O 'Geospatial_Coordinates.csv'  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print('Data downloaded!')

Data downloaded!


In [16]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')

#### Joining two Dataframes on postal code to get coordinates

In [17]:
final_df = pd.merge(df, geo_df, left_on='Postcode', right_on='Postal Code')

In [18]:
final_df=final_df[['Borough','Postcode','Neighbourhood','Address','Latitude','Longitude']]

#### Filter out only Toronto records

In [19]:
toronto_df = final_df[final_df['Borough'].str.contains("Toronto")]

In [20]:
toronto_df.head(50)

,Borough,Postcode,Neighbourhood,Address,Latitude,Longitude
2,Downtown Toronto,M5A,"Harbourfront, Regent Park","Downtown Toronto, M5A",43.654260,-79.360636
9,Downtown Toronto,M5B,"Ryerson, Garden District","Downtown Toronto, M5B",43.657162,-79.378937
15,Downtown Toronto,M5C,St. James Town,"Downtown Toronto, M5C",43.651494,-79.375418
19,East Toronto,M4E,The Beaches,"East Toronto, M4E",43.676357,-79.293031
20,Downtown Toronto,M5E,Berczy Park,"Downtown Toronto, M5E",43.644771,-79.373306
24,Downtown Toronto,M5G,Central Bay Street,"Downtown Toronto, M5G",43.657952,-79.387383
25,Downtown Toronto,M6G,Christie,"Downtown Toronto, M6G",43.669542,-79.422564
30,Downtown Toronto,M5H,"Adelaide, King, Richmond","Downtown Toronto, M5H",43.650571,-79.384568
31,West Toronto,M6H,"Dovercourt Village, Dufferin","West Toronto, M6H",43.669005,-79.442259
36,Downtown Toronto,M5J,"Harbourfront East, Toronto Islands, Union Station","Downtown Toronto, M5J",43.640816,-79.381752


In [21]:
toronto_addr = toronto_df[toronto_df.Address == 'Downtown Toronto, M5A']

latitude = toronto_addr['Latitude'].values.tolist()
longitude = toronto_addr['Longitude'].values.tolist()

latitude = latitude[0]
longitude = longitude[0]

In [22]:

# create map of Toront using and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format('Neighbourhood', 'Borough')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
   
map_toronto

In [23]:
CLIENT_ID = 'AO2QOPQFBXPQICU40CAJVV3NHDYQBYOBW2UPGZQ2FWH2TM1R' # your Foursquare ID
CLIENT_SECRET = 'J5IKSD5VELGKHWZIFOWXPJA43XOHICXCMQHEK3OTE3NBCV2R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AO2QOPQFBXPQICU40CAJVV3NHDYQBYOBW2UPGZQ2FWH2TM1R
CLIENT_SECRET:J5IKSD5VELGKHWZIFOWXPJA43XOHICXCMQHEK3OTE3NBCV2R


In [24]:
toronto_df.loc[24, 'Neighbourhood']


'Central Bay Street'

In [25]:
neighborhood_latitude = toronto_df.loc[24, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[24, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[24, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


In [26]:
LIMIT = 100 
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=AO2QOPQFBXPQICU40CAJVV3NHDYQBYOBW2UPGZQ2FWH2TM1R&client_secret=J5IKSD5VELGKHWZIFOWXPJA43XOHICXCMQHEK3OTE3NBCV2R&v=20180605&ll=43.6579524,-79.3873826&radius=500&limit=100'

In [27]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [28]:
results = requests.get(url).json()

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Hailed Coffee,Coffee Shop,43.658833,-79.383684
3,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524
4,Mercatto,Italian Restaurant,43.660391,-79.387664


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

85 venues were returned by Foursquare.


#### 2. Explore Neighbourhoods in Toronto downtown

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:

toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(1702, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


In [35]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",52,52,52,52,52,52
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


Let's find out how many unique categories can be curated from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


#### 3. Now Analyzing Each neighbourhood

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_onehot.shape

(1702, 234)

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()


#### top 5 most common venues

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2                  Bar  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0         Coffee Shop  0.13
1      Breakfast Spot  0.09
2                Café  0.09
3           Nightclub  0.09
4  Italian Restaurant  0.04


----Business reply mail Processing Centre969 Eastern----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2         Garden Center  0.06
3                Garden  0.06
4  Fast Food Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  fre

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Restaurant,Gym,Hotel,Japanese Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Steakhouse,Restaurant,Cheese Shop,Café
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Nightclub,Café,Breakfast Spot,Grocery Store,Furniture / Home Store,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Garden,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place,Restaurant,Recording Studio,Auto Workshop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Boutique
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Indian Restaurant,Bakery,Chinese Restaurant,Park,Pizza Place
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Japanese Restaurant,Ice Cream Shop,Burger Joint,Bar,Indian Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Bakery,Vietnamese Restaurant,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Hotpot Restaurant
8,Christie,Grocery Store,Café,Park,Baby Store,Convenience Store,Coffee Shop,Athletics & Sports,Restaurant,Diner,Nightclub
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Burger Joint,Gay Bar,Sushi Restaurant,Restaurant,Bubble Tea Shop,Mediterranean Restaurant,Men's Store,Gastropub


#### 4. Cluster Neighborhoods

In [43]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [44]:
toronto_merged = toronto_df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,Postcode,Neighbourhood,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,M5A,"Harbourfront, Regent Park","Downtown Toronto, M5A",43.654260,-79.360636,2,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Pub,Mexican Restaurant,Hotel,Chocolate Shop,Greek Restaurant
9,Downtown Toronto,M5B,"Ryerson, Garden District","Downtown Toronto, M5B",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Restaurant,Plaza,Bubble Tea Shop,Italian Restaurant
15,Downtown Toronto,M5C,St. James Town,"Downtown Toronto, M5C",43.651494,-79.375418,2,Coffee Shop,Restaurant,Café,Clothing Store,Hotel,Gastropub,Cosmetics Shop,Bakery,Japanese Restaurant,Italian Restaurant
19,East Toronto,M4E,The Beaches,"East Toronto, M4E",43.676357,-79.293031,2,Pub,Coffee Shop,Other Great Outdoors,Neighborhood,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
20,Downtown Toronto,M5E,Berczy Park,"Downtown Toronto, M5E",43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Steakhouse,Restaurant,Cheese Shop,Café


Finally, let's visualize the resulting clusters

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 5. Examine each cluster 

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

##### Cluster 1 Commercial - Downtown Toronto (Eatery)

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,M6K,-79.428191,0,Coffee Shop,Nightclub,Café,Breakfast Spot,Grocery Store,Furniture / Home Store,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant
97,M5X,-79.382280,0,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Gastropub,Gym,Deli / Bodega,Japanese Restaurant


Cluster 2 Central Toronto - Residential 

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,M4N,-79.388790,1,Gym / Fitness Center,Bus Line,Park,Swim School,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
75,M6R,-79.456325,1,Coffee Shop,Gift Shop,Dessert Shop,Cuban Restaurant,Dog Run,Movie Theater,Burger Joint,Bar,Restaurant,Breakfast Spot
81,M6S,-79.484450,1,Coffee Shop,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Restaurant,Gastropub,Food,Latin American Restaurant,Sandwich Place


Cluster 3 Downtown - Underground city
It is very interesting that it was able to separate it in different cluster.

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,-79.360636,2,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Pub,Mexican Restaurant,Hotel,Chocolate Shop,Greek Restaurant
9,M5B,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Restaurant,Plaza,Bubble Tea Shop,Italian Restaurant
15,M5C,-79.375418,2,Coffee Shop,Restaurant,Café,Clothing Store,Hotel,Gastropub,Cosmetics Shop,Bakery,Japanese Restaurant,Italian Restaurant
19,M4E,-79.293031,2,Pub,Coffee Shop,Other Great Outdoors,Neighborhood,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
20,M5E,-79.373306,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Steakhouse,Restaurant,Cheese Shop,Café
24,M5G,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Japanese Restaurant,Ice Cream Shop,Burger Joint,Bar,Indian Restaurant
25,M6G,-79.422564,2,Grocery Store,Café,Park,Baby Store,Convenience Store,Coffee Shop,Athletics & Sports,Restaurant,Diner,Nightclub
30,M5H,-79.384568,2,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,American Restaurant,Restaurant,Gym,Hotel,Japanese Restaurant
31,M6H,-79.442259,2,Bakery,Supermarket,Fast Food Restaurant,Music Venue,Brewery,Bar,Bank,Discount Store,Middle Eastern Restaurant,Café
36,M5J,-79.381752,2,Coffee Shop,Hotel,Café,Pizza Place,Aquarium,Brewery,Italian Restaurant,Sports Bar,Scenic Lookout,Fried Chicken Joint


Cluster 4 - Residential classification with transportation option (Lawrence Park, Runnymede, Swansea)

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,M4T,-79.38316,3,Gym,Park,Playground,Trail,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 5- Old Toronto - Mixed density (North toronto west)

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,M4R,-79.405678,4,Sporting Goods Shop,Coffee Shop,Yoga Studio,Fast Food Restaurant,Dessert Shop,Metro Station,Salon / Barbershop,Sandwich Place,Café,Diner
